In [ ]:
# Importamos las lbrerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Lectura con pandas de los archivos CSV 
df_1 = pd.read_csv('str_1.csv', delimiter=';')
df_2 = pd.read_csv('str_2.csv', delimiter=';')

df = pd.concat([df_1, df_2], ignore_index=True)

In [ ]:
# Observamos los primeros registros del dataframe
df.head()

In [ ]:
# Eliminamos la columna ID
df.drop('ID', axis=1, inplace=True)

In [ ]:
# Verificamos que se elimino la columna ID
df.head()

## VALORES ERRÓNEOS Y/O FALTANTES

In [ ]:
# Observamos los valores faltantes
df.isnull().sum()

In [ ]:
# Asignar a los valores faltantes de intentos de login la mediana ya que son numeros naturales
df['login_attempts'].fillna(df['login_attempts'].median(), inplace=True)

In [ ]:
# Protocolo de encriptación
df_encryption_used = pd.DataFrame(df['encryption_used'])
df_encryption_used.fillna('Vacio', inplace=True)

plt.pie(df_encryption_used['encryption_used'].value_counts(), labels=df_encryption_used['encryption_used'].unique(), autopct='%1.2f%%') #
plt.title('Proporción de distintos protocolos de encriptación usados')

plt.tight_layout()
plt.show()

In [ ]:
# Para los datos faltantes del protocolo de encriptación
# Obtener distribución original (sin NaN)
dist = df['encryption_used'].value_counts(normalize=True)

# Generar valores aleatorios con esa distribución
missing_count = df['encryption_used'].isna().sum()
imputed_values = np.random.choice(dist.index, size=missing_count, p=dist.values)

# Reemplazar NaNs con los valores generados
df.loc[df['encryption_used'].isna(), 'encryption_used'] = imputed_values

# Si la variable tiene correlación con otras podrías entrenar un modelo de clasificación

In [ ]:
# Protocolo de encriptación
plt.pie(df['encryption_used'].value_counts(), labels=df['encryption_used'].unique(), autopct='%1.2f%%') #
plt.title('Proporción de distintos protocolos de encriptación usados')

plt.tight_layout()
plt.show()

In [ ]:
# Verificamos los valores faltantes
df.isnull().sum()

## VALORES ATÍPICOS

In [ ]:
# Boxplot para visualizar valores atípicos
# Visualizamos la distribución de las distintas variables
fig, ax = plt.subplots(figsize=(15,6))
df.plot(kind='box', ax=ax)

## PROCESAMIENTO DE VARIABLES CATEGÓRICAS

In [ ]:
df = pd.get_dummies(df, columns=['protocol','browser_type'], dtype=int)
df = pd.get_dummies(df, columns=['encryption_used', 'unusual_time_access'], dtype=int)

In [ ]:
df.head()

Los valores de la variable categórica que representa nuestras etiquetas son Strings 'Y' y 'N' representando fue atacado o no. Cambiamos estos resultados por 1 para la etiqueta 'Y' y 0 para la etiqueta 'N'.

In [ ]:
# Visualizar los valores de la columna (unique).
df['attack'].unique()

In [ ]:
# Cambiar los valores Y y N de la columna "attack" por 1 y 0
df['attack'] = df['attack'].map({'Y':1,'N':0})

In [ ]:
# Visualizar los valores de la columna (unique).
df['attack'].unique()

## BALANCE DEL CONJUNTO DE DATOS

In [ ]:
# Visualizar el balance del dataset
df['attack'].value_counts()

In [ ]:
# Visualización grafica
df['attack'].value_counts().plot(kind='bar')
plt.title('Distribución de clases en attack')
plt.xlabel('Clase')
plt.ylabel('Cantidad de muestras')
plt.xticks([0, 1], ['0 (Normal)', '1 (Ataque)'], rotation=0)
plt.show()

In [ ]:
# Importamos la libreira necesaria
from imblearn.over_sampling import SMOTE

In [ ]:
# Nos guradamos las etiquetas en un vector auxiliar
attack = df['attack']

# Aplicamos el over_sampling de los datos

df, attack = SMOTE().fit_resample(df, attack)

In [ ]:
# Visualización grafica
df['attack'].value_counts().plot(kind='bar')
plt.title('Distribución de clases en attack')
plt.xlabel('Clase')
plt.ylabel('Cantidad de muestras')
plt.xticks([0, 1], ['0 (Normal)', '1 (Ataque)'], rotation=0)
plt.show()

In [ ]:
df['attack'].value_counts()

Otro

In [ ]:
# Importar el paquete imblearn
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Separar features (X) y la variable objetivo (y)
X = df.drop('attack', axis=1)
y = df['attack']

In [ ]:
# Visualizar el balance del dataset
sns.countplot(x=y)

In [ ]:
# Generar un nuevos conjunto de datos balanceado por Over-sampling
# Definimos la estrategia de Oversampling, la clase minoritaria tendrá la misma cantidad que la mayoritaria.
oversample = RandomOverSampler(sampling_strategy='minority')

In [ ]:
# Generamos el nuevo dataset balanceado
X_over, y_over = oversample.fit_resample(X, y)

In [ ]:
# Visualizar el balance del dataset generado
sns.countplot(x=y_over)

## NORMALIZACIÓN DEL CONJUNTO DE DATOS

In [ ]:
# Selección de variables numéricas
num_cols = ['login_attempts', 'duration', 'packet_size', 'ip_reputation_score', 'failed_logins']


# Filtramos los datos
data_numeric = df[num_cols]


# Calculamos la media y desviación estándar
means = data_numeric.mean()
stds = data_numeric.std()


# Aplicamos la normalización Z-score
data_scaled = (data_numeric - means) / stds


# Reemplazamos en el DataFrame original
# df_scaled = df.copy()
# df_scaled[num_cols] = data_scaled
df[num_cols] = data_scaled

# **Modelado**

In [ ]:
# Carga de los paquetes necesarios
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# Visualizar las columnas del dataframe
df.columns

In [ ]:
# Seleccionar las columnas del dataset "df" que corresponden a las entradas del modelo y la salida esperada.
X = df[['login_attempts', 'duration', 'packet_size', 'ip_reputation_score',
       'failed_logins', 'attack', 'protocol_ICMP', 'protocol_TCP',
       'protocol_UDP', 'browser_type_Chrome', 'browser_type_Edge',
       'browser_type_Firefox', 'browser_type_Safari', 'browser_type_Unknown',
       'encryption_used_AES', 'encryption_used_DES', 'unusual_time_access_0',
       'unusual_time_access_1']]
y = df['attack'] 

# **Modelos de clasificació**

In [ ]:
X.head()

## Partición del conjunto de datos

In [ ]:
# Separar el conjunto de datos en una particion para entrenamiento y una para test con el 30% del conjunto.
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.3)

In [ ]:
# Visualizar el resultado de la función (shape)
print(X.shape, X_train.shape, X_test.shape)
print(y.shape, y_train.shape, y_test.shape)

## Árbol de Decisión

In [ ]:
# Instanciar un Arbol de Decisión
model_Tree = DecisionTreeClassifier(max_features=6, max_depth=6)

In [ ]:
# Entrenar el modelo con el conjunto de entrenamiento
model_Tree.fit(X_train, y_train)

In [ ]:
# Calculando la precisión para el conjunto de entrenamiento
X_train_prediction = model_Tree.predict(X_train)
train_accuracy = accuracy_score(X_train_prediction, y_train)
train_f1 = f1_score(X_train_prediction, y_train)
print('Accuracy para los datos de entrenamiento : ', train_accuracy)
print('Precision para los datos de entrenamiento : ', train_f1)

In [ ]:
# Calculando la precisión para el conjunto de test
X_test_prediction = model_Tree.predict(X_test)
test_accuracy = accuracy_score(X_test_prediction, y_test)
test_f1 = f1_score(X_test_prediction, y_test)
print('Accuracy para los datos de test : ', test_accuracy)
print('Precision para los datos de test : ', test_f1)